In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm_notebook

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 3, padding=1,bias=False) #input 28 OUtput26 RF3
        self.bn1   = nn.BatchNorm2d(10)
        self.dropout1 = nn.Dropout2d(0.02)
        self.conv2 = nn.Conv2d(10, 10, 3, padding=1,bias=False)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(10, 20, 3, padding=1,bias=False)
        self.conv4 = nn.Conv2d(20, 40, 3)
        self.conv5 = nn.Conv2d(40, 10, 3)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv6 = nn.Conv2d(10, 10, 3,padding=1,bias=False)
        self.bn6   = nn.BatchNorm2d(10)        
        self.dropout2 = nn.Dropout2d(0.02)

        self.conv7 = nn.Conv2d(10,10, 3) #input 5, Output 3, RF: 25 
        self.bn7   = nn.BatchNorm2d(10)
        self.conv8 = nn.Conv2d(10, 10, 3)
        self.avgpool = nn.AvgPool2d((1,1))
       

    def forward(self, x):
        #print("x1 ",x.shape)
        x = self.pool1(F.relu(self.conv2(self.dropout1(self.bn1(F.relu(self.conv1(x)))))))
        #print("x2 ",x.shape)
        x = F.relu(self.conv3(x))
        #print("x3 ",x.shape)
        x = self.dropout2(self.bn6(self.conv6(self.pool2(F.relu(self.conv5(F.relu(self.conv4(x))))))))
        #print("x4 ",x.shape)
        x=self.conv8(self.bn7(F.relu(self.conv7(x))))
        x=self.avgpool(x)
        #print("x5 ",x.shape)
        x = x.view(-1, 10)
        #print("x6 ",x.shape)
        return F.log_softmax(x)
        

In [26]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]              90
       BatchNorm2d-2           [-1, 10, 28, 28]              20
         Dropout2d-3           [-1, 10, 28, 28]               0
            Conv2d-4           [-1, 10, 28, 28]             900
         MaxPool2d-5           [-1, 10, 14, 14]               0
            Conv2d-6           [-1, 20, 14, 14]           1,800
            Conv2d-7           [-1, 40, 12, 12]           7,240
            Conv2d-8           [-1, 10, 10, 10]           3,610
         MaxPool2d-9             [-1, 10, 5, 5]               0
           Conv2d-10             [-1, 10, 5, 5]             900
      BatchNorm2d-11             [-1, 10, 5, 5]              20
        Dropout2d-12             [-1, 10, 5, 5]               0
           Conv2d-13             [-1, 10, 3, 3]             910
      BatchNorm2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    correct = 0
    pbar = tqdm_notebook(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

    print('\nTraining Set: Accuracy: {}/{} ({:.2f}%)\n'.format(
        correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))      


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [29]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)

for epoch in range(1, 20):
    print("Epoch Number:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Epoch Number: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.




Training Set: Accuracy: 57250/60000 (95.42%)


Test set: Average loss: 0.0579, Accuracy: 9828/10000 (98.28%)

Epoch Number: 2




Training Set: Accuracy: 59041/60000 (98.40%)


Test set: Average loss: 0.0405, Accuracy: 9873/10000 (98.73%)

Epoch Number: 3




Training Set: Accuracy: 59271/60000 (98.78%)


Test set: Average loss: 0.0457, Accuracy: 9859/10000 (98.59%)

Epoch Number: 4


...



Training Set: Accuracy: 59819/60000 (99.70%)


Test set: Average loss: 0.0221, Accuracy: 9937/10000 (99.37%)

Epoch Number: 16




Training Set: Accuracy: 59844/60000 (99.74%)


Test set: Average loss: 0.0226, Accuracy: 9937/10000 (99.37%)

Epoch Number: 17




Training Set: Accuracy: 59872/60000 (99.79%)


Test set: Average loss: 0.0215, Accuracy: 9943/10000 (99.43%)

Epoch Number: 18




Training Set: Accuracy: 59863/60000 (99.77%)


Test set: Average loss: 0.0231, Accuracy: 9929/10000 (99.29%)

Epoch Number: 19




Training Set: Accuracy: 59874/60000 (99.79%)


Test set: Average loss: 0.0234, Accuracy: 9933/10000 (99.33%)

